In [1]:
import json
import os

def convert_labelme_to_yolo(json_path, output_labels_dir, images_base_dir, class_name="object"):
    # Read JSON file
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {json_path}. Skipping.")
            return

    # Extract image filename from JSON
    image_filename = data.get("imagePath", "")
    if not image_filename:
        print(f"Skipping {json_path}: 'imagePath' field is missing.")
        return

    # Get just the filename (remove any directory paths)
    image_filename = os.path.basename(image_filename)

    # Check where the image exists (train or valid)
    train_image_path = os.path.join(images_base_dir, "train", image_filename)
    valid_image_path = os.path.join(images_base_dir, "valid", image_filename)

    if os.path.exists(train_image_path):
        split = "train"
    elif os.path.exists(valid_image_path):
        split = "valid"
    else:
        print(f"Skipping {json_path}: Corresponding image not found in train/valid.")
        return

    # Get image dimensions
    try:
        img_width = data["imageWidth"]
        img_height = data["imageHeight"]
    except KeyError:
        print(f"Skipping {json_path}: Missing image dimensions.")
        return

    # Process shapes
    yolo_lines = []
    for shape in data.get("shapes", []):
        label = shape.get("label", class_name)
        points = shape.get("points", [])

        if len(points) < 2:
            continue  # Skip invalid shapes

        # Get bounding box coordinates
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)

        # Convert to YOLO format
        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        yolo_lines.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Create output directory if needed
    output_split_dir = os.path.join(output_labels_dir, split)
    os.makedirs(output_split_dir, exist_ok=True)

    # Save YOLO txt file
    txt_filename = os.path.join(
        output_split_dir,
        os.path.basename(json_path).replace(".json", ".txt")
    )

    with open(txt_filename, "w") as f:
        f.writelines(yolo_lines)
    print(f"Converted: {json_path} -> {txt_filename}")

# Configuration
json_folder = "C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels"
output_labels_dir = "C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels"
images_base_dir = "C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/images"

# Create output directories if they don't exist
os.makedirs(output_labels_dir, exist_ok=True)

# Process all JSON files
for json_file in os.listdir(json_folder):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_folder, json_file)
        convert_labelme_to_yolo(
            json_path=json_path,
            output_labels_dir=output_labels_dir,
            images_base_dir=images_base_dir
        )

Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels\train\pp.txt
Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp10.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels\train\pp10.txt
Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp11.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels\train\pp11.txt
Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp12.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels\train\pp12.txt
Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp13.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels\train\pp13.txt
Converted: C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/Json_labels\pp14.json -> C:/Users/aksha/OneDrive/Documents/vizexpert/labelme/dataset/labels